In [1]:
minmaxInterval = 2000

## open port

In [2]:
!echo 'quzinee1!'| sudo -S chmod a+rw /dev/ttyUSB1

[sudo] password for terry: chmod: cannot access '/dev/ttyUSB1': No such file or directory


## BASIC USAGE OF XL320 CLASS

In [3]:
import os, time
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from math import pi as PI
DEG90 = PI/2
PROPORTION_MATRIX = np.array(
    [
        [1,0.1,0.4,0.4, 0,0,0,0, 0,0,0,0, 0,0,0,0],
        [0.1,1,0.4,0.4, 0,0,0,0, 0,0,0,0, 0,0,0,0],
        [0.4,0.4,1,0.1, 0,0,0,0, 0,0,0,0, 0,0,0,0],
        [0.4,0.4,0.1,1, 0,0,0,0, 0,0,0,0, 0,0,0,0],
        
        [0,0,0,0,  1,0.1,0.4,0.4, 0,0,0,0, 0,0,0,0],
        [0,0,0,0,  0.1,1,0.4,0.4,  0,0,0,0, 0,0,0,0],
        [0,0,0,0,  0.4,0.4,1,0.1, 0,0,0,0, 0,0,0,0],
        [0,0,0,0,  0.4,0.4,0.1,1, 0,0,0,0, 0,0,0,0],
        
        [0,0,0,0, 0,0,0,0, 1,0.1,0.4,0.4, 0,0,0,0],
        [0,0,0,0, 0,0,0,0, 0.1,1,0.4,0.4, 0,0,0,0],
        [0,0,0,0, 0,0,0,0, 0.4,0.4,1,0.1, 0,0,0,0],
        [0,0,0,0, 0,0,0,0, 0.4,0.4,0.1,1, 0,0,0,0],
        
        [0,0,0,0, 0,0,0,0, 0,0,0,0, 1,0.1,0.4,0.4],
        [0,0,0,0, 0,0,0,0, 0,0,0,0, 0.1,1,0.4,0.4],
        [0,0,0,0, 0,0,0,0, 0,0,0,0, 0.4,0.4,1,0.1],
        [0,0,0,0, 0,0,0,0, 0,0,0,0, 0.4,0.4,0.1,1],
        ])
PROPORTION_MATRIX = PROPORTION_MATRIX[:4,:4]
PROPORTION_MATRIX.shape

(4, 4)

In [4]:
import sys
sys.path.append(".")
""" FOR DYNAMIXEL """
from model.class_xc330 import xc330
""" FOR MODERN DRIVER """
import roslib; roslib.load_manifest('ur_driver')
import rospy
import actionlib
from control_msgs.msg import *
from trajectory_msgs.msg import *
from sensor_msgs.msg import JointState

from class_vicon_marker import VICON
from class_vicon_base import ViconBase

class OneFinger(object):
    def __init__(self,):
        rospy.init_node("DEMO")
        self.JOINT_NAMES = ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint',
                            'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
        self.client      = None
        
        self.xc          = xc330('SOFTROBOT', _USB_NUM=0)
        
    def init_xc(self,initpos):
        self.xc.connect()
        self.xc.set_operatingmode([4])

        self.xc.set_torque([0])
        time.sleep(2)
        self.xc.set_torque([1])
        print ("TORQUE ON")

        currpos = self.xc.get_currpos()
        minmaxInterval = 5000
        self.xc.set_minmaxpos(currpos-np.ones_like(currpos)*minmaxInterval,currpos+np.ones_like(currpos)*minmaxInterval)

        if initpos is None:
            initpos = currpos; print("No initpos Found")
            self.set_init_pos()
            return currpos

        self.xc.set_goalposcluster(initpos,1)

        print("INITIALIZE POSITION")
        currpos = self.xc.get_currpos()
        print(currpos)

        minmaxInterval = minmaxInterval
        self.xc.set_minmaxpos(currpos-np.ones_like(currpos)*minmaxInterval,currpos+np.ones_like(currpos)*minmaxInterval)
        self.set_init_pos()
        return currpos
        
    def set_init_pos(self):
        self.initpos = self.xc.get_currpos()
    # def move_xc(self,pos):
    #     pos = self.initpos + np.array(pos)* np.array([1,-1,-1,1])
    #     self.xc.set_goalposcluster(pos,1)
        
    def pmove_xc(self,pos):
        pos = self.initpos + PROPORTION_MATRIX @ np.array(pos) * np.array([1,-1,-1,1])
        self.xc.set_goalposcluster(pos,1)
        
    def move_arm_speed(self, joints, speed):
        try: 
            q = joints
            g = FollowJointTrajectoryGoal()
            g.trajectory = JointTrajectory()
            g.trajectory.joint_names = self.JOINT_NAMES
            joint_states = rospy.wait_for_message("joint_states", JointState)
            joints_pos   = joint_states.position

            diff = np.array(joints_pos) - np.array(q)

            assert speed <= 1e-1 
            movetime = np.linalg.norm(diff)/speed

            g.trajectory.points = [
                JointTrajectoryPoint(positions=joints_pos, velocities=[0]*6, time_from_start=rospy.Duration(0.0)),
                JointTrajectoryPoint(positions=q, velocities=[0]*6, time_from_start=rospy.Duration(movetime))]  

            self.client.send_goal(g)
            self.client.wait_for_result()
        except KeyboardInterrupt:
            self.client.cancel_goal()
            raise
        except:
            raise  
        
    def move_xc(self, control, movetime):
        controls = np.array(control).astype(int)
        assert controls.ndim == 1

        
        pos_list = []
        currpos = self.xc.get_currpos()
        goal_pos  = control * np.array([1,-1,-1,1]) + self.initpos

        resolution = 5
        splitnum = np.linalg.norm(goal_pos - currpos)/resolution

        interpolate_pos = np.linspace(currpos,goal_pos,int(splitnum))
        currpos = goal_pos
        pos_list.append(interpolate_pos)
        
        pos_array = np.vstack(pos_list)

        if len(pos_array) > 0:
            unit_sleep = movetime/len(pos_array)
            for pos in pos_array:
                self.xc.set_goalpos(pos)
                time.sleep(unit_sleep)


    def multiple_move_xc(self, controls, movetimes):
        controls = np.array(controls).astype(int)
        movetimes = np.array(movetimes)
        assert controls.ndim == 2
        assert movetimes.ndim == 1
        
        pos_list = []
        currpos = self.xc.get_currpos()

        for control in controls:
            goal_pos  = control * np.array([1,-1,-1,1]) + self.initpos

            resolution = 5
            splitnum = np.linalg.norm(goal_pos - currpos)/resolution

            interpolate_pos = np.linspace(currpos,goal_pos,int(splitnum))
            currpos = goal_pos
            pos_list.append(interpolate_pos)
            
        # pos_array = np.vstack(pos_list)
        for pos, movetime in zip(pos_list, movetimes):
            unit_sleep = movetime/len(pos)
            for pos_ in pos:
                self.xc.set_goalpos(pos_)
                time.sleep(unit_sleep)

    def execute_arm_speed(self, joints, speed):
        try:
            self.client = actionlib.SimpleActionClient('follow_joint_trajectory', FollowJointTrajectoryAction)
            print("Waiting for server...")
            self.client.wait_for_server()
            print("Connected to server")
            """ Initialize """
            self.move_arm_speed(joints, speed)
            print("Finish plan")

        except KeyboardInterrupt:
            rospy.signal_shutdown("KeyboardInterrupt")
            raise      


['/home/terry/Rilab/sOftrobot/SORO-Dynamixel-python', '', '/home/terry/catkin_ws/devel/lib/python3/dist-packages', '/opt/ros/noetic/lib/python3/dist-packages', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '/home/terry/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '.']


## INSTANTIATE Robot 

In [5]:
robot = OneFinger()

[SOFTROBOT] INSTANTIATED AT [b'/dev/ttyUSB0']


In [39]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90, 0]
robot.execute_arm_speed(init_joint, speed=1e-1)

Waiting for server...
Connected to server
Finish plan


In [7]:
currpos = robot.init_xc([4722, 3082,  957, 1628])

[SOFTROBOT] CONNECT
 SUCCEEDED TO OPEN THE PORT!
 SUCCEEDED TO CHANGE THE BAUDRATE!
COMMUNICATION SUCCESS
[SOFTROBOT] DETECT ID
[SOFTROBOT] [4]MOTORS DETECTED 
[SOFTROBOT] ID_LIST:[[13, 14, 15, 16]]
TORQUE ON
INITIALIZE POSITION
[4682 3096  977 1595]


In [8]:
# robot.move_xc([1000,0,0,0], movetime=5)

In [9]:
def move_arm_soro(robot:OneFinger, qs_array, motor_array, time_array):
    qs_array = np.array(qs_array)
    motor_array = np.array(motor_array)

    assert qs_array.ndim == 2
    assert motor_array.ndim == 2
    assert len(time_array) == len(qs_array)
    assert len(time_array) == len(motor_array)

    motor_list = []
    currpos = robot.xc.get_currpos()
    
    for motor_control in motor_array:
        goal_pos  = motor_control * np.array([1,-1,-1,1]) + robot.initpos

        resolution = 10
        splitnum = np.linalg.norm(goal_pos - currpos)/resolution

        interpolate_pos = np.linspace(currpos,goal_pos,int(splitnum))
        currpos = goal_pos
        motor_list.append(interpolate_pos)
    
    from tqdm.notebook import tqdm
    i = 0
    for motor_array, q, movetime in tqdm(zip(motor_list, qs_array, time_array)):
        
        try:
            g = FollowJointTrajectoryGoal()
            g.trajectory = JointTrajectory()
            g.trajectory.joint_names = robot.JOINT_NAMES
            joint_states = rospy.wait_for_message("joint_states", JointState)
            joints_pos   = joint_states.position

            diff = np.array(joints_pos) - np.array(q)

            speed = np.linalg.norm(diff)/movetime

            speed_max = 1e-1
            if speed > speed_max:
                movetime = np.linalg.norm(diff)/speed_max
            
            
            # g.trajectory.points = [
            #     JointTrajectoryPoint(positions=joints_pos, velocities=[0]*6, time_from_start=rospy.Duration(0.0)),
            #     JointTrajectoryPoint(positions=q, velocities=[0]*6, time_from_start=rospy.Duration(movetime))
            #     ]  
            
            qs = np.linspace(joints_pos, q, 100)
            temp = []
            for j,q_ in enumerate(qs):
                temp.append(JointTrajectoryPoint(positions=q_, velocities=[0]*6, time_from_start=rospy.Duration((j-1)*movetime/len(qs))))
            
            g.trajectory.points = temp

            print(q)
            robot.client.send_goal(g)
            unit_sleep = movetime/len(motor_array)
            for motor in motor_array:
                robot.xc.set_goalpos(motor)
                time.sleep(unit_sleep* 0.7)
            robot.client.wait_for_result()

        except KeyboardInterrupt:
            robot.client.cancel_goal()
            raise
        except:
            raise  

        robot.client.wait_for_result()
        




# get qs and motor for paint

In [10]:
q0 = [0,-DEG90,DEG90,-DEG90,-DEG90, 0]
motor0 = [0,0,0,0]

q1 = np.array([-0.6, -57.6, 109.6, -179.4, -100.5, 0.0])/180 * PI
motor1 = [0,1000,0,0]
q2 = np.array([-3.6, -56.0, 108.3, -177.1, -97.4, 0.0])/180 * PI
moto2 = [1000,0,0,0]

qs_array = np.array([q0,q1,q2])
motor_array = np.array([motor0,motor1,moto2])

qs_array.shape, motor_array.shape

((3, 6), (3, 4))

In [11]:
move_arm_soro(robot, qs_array, motor_array, time_array=2*np.ones(len(motor_array)).astype(np.float32))

0it [00:00, ?it/s]

[ 0.         -1.57079633  1.57079633 -1.57079633 -1.57079633  0.        ]
[-0.01047198 -1.00530965  1.91288086 -3.13112068 -1.7540559   0.        ]
[ 0.06283185 -0.97738438  1.89019158 -3.09097811 -1.69995069  0.        ]


In [37]:
pertub = [2,3,0,1]

motor_array = np.load("planned_traj/paint/motor_array.npy").astype(np.float32)[:, pertub]
qs_array    = np.load("planned_traj/paint/qs_array.npy").astype(np.float32)

assert (qs_array <  PI).all() and (qs_array>-PI).all()

In [38]:
move_arm_soro(robot, qs_array, motor_array, time_array=2 * np.ones(len(motor_array)).astype(np.float32))

0it [00:00, ?it/s]

[ 0.        -1.5707964  1.5707964 -1.5707964 -1.5707964  0.       ]
[ 0.00402592 -1.5481125   1.6148673  -1.6375489  -1.5707964   0.00402654]
[-0.01470772 -1.4827226   1.6207126  -1.7087858  -1.5707964  -0.01470811]
[-0.02856332 -1.4114763   1.6138504  -1.7731702  -1.5707964  -0.02856405]
[-0.04066654 -1.3356045   1.5974509  -1.8326427  -1.5707963  -0.04066747]
[-0.05153741 -1.2559649   1.5710888  -1.8859208  -1.5707961  -0.05153854]
[-0.06136599 -1.1730137   1.5344644  -1.9322476  -1.570796   -0.06136735]
[-0.07026374 -1.0870807   1.4872328  -1.9709498  -1.5707959  -0.07026538]
[-0.07830586 -0.9983497   1.4289579  -2.0014064  -1.5707959  -0.07830779]
[-0.08555821 -0.90674466  1.358935   -2.0229888  -1.5707958  -0.08556052]
[-0.08555821 -0.90674466  1.358935   -2.0229888  -1.5707958  -0.08556052]


/tmp/ipykernel_36723/1142653806.py:57: RuntimeWarning: divide by zero encountered in divide
  unit_sleep = movetime/len(motor_array)


[-0.08968499 -0.9063684   1.3601892  -2.024618   -1.570796   -0.08968574]
[-0.09487383 -0.909061    1.3647705  -2.0265057  -1.5707963  -0.09487335]
[-0.09993018 -0.9138848   1.3715167  -2.0284271  -1.5707964  -0.09992886]
[-0.10453302 -0.9218143   1.381792   -2.030772   -1.5707965  -0.1045312 ]
[-0.09800417 -0.94581103  1.4145575  -2.0395415  -1.5707966  -0.09800343]
[-0.09921987 -0.94820756  1.4186143  -2.0412014  -1.5707966  -0.09921865]
[-0.10344716 -0.9533125   1.427383   -2.0448647  -1.5707967  -0.10344516]
[-0.10861161 -0.9592518   1.4379708  -2.0495136  -1.5707968  -0.10860988]
[-0.11064094 -0.96423554  1.4483029  -2.0548625  -1.5707968  -0.11064015]


In [8]:
q0 = init_joint
q1 = np.array([0, -44, 63, -139, -92, 0]) / 180 * PI
q2 = np.array([0, -33, 58, -150, -92, 0])/180 * PI
q3 = np.array([0, -76, 124, -150, -93, 11])/180 * PI

motor_0 = [0,0,0,0]
motor_1 = [0,0,0,1000]
motor_2 = [0,0,1000,0]
motor_3 = [0,0,1000,0]


In [30]:
qs_array = np.array([q0, q1, q2, q3, q0])
motor_array = np.array([motor_0, motor_1, motor_2, motor_3, motor_0])
time_array = np.array([1,1,3,1,1])
# move_arm_soro(robot, qs_array, motor_array, time_array=time_array)

qs_array.shape, motor_array.shape,


((5, 6), (5, 4))

In [31]:
motor_array[:,[2,3,0,1]]

# 1 -> 3
# 2 -> 4
# 3 -> 1
# 4 -> 2

array([[   0,    0,    0,    0],
       [   0, 1000,    0,    0],
       [1000,    0,    0,    0],
       [1000,    0,    0,    0],
       [   0,    0,    0,    0]])

In [32]:
pertub = [2,3,0,1]

motor_array = np.load("planned_traj/scraps/motor_array.npy")[:,0,:].astype(np.float32)[:, pertub]
qs_array = np.load("planned_traj/scraps/qs_array.npy").astype(np.float32)

qs_array.shape, motor_array.shape

((30, 6), (30, 4))

In [33]:
move_arm_soro(robot, qs_array, motor_array, time_array=np.ones(len(motor_array)).astype(np.float32))

0it [00:00, ?it/s]

[-0.04599259 -1.5802035   1.6107767  -1.5999597  -1.56946    -0.04463042]
[-0.03281507 -1.519118    1.6044059  -1.7096051  -1.5721114  -0.03104687]
[-0.0224144  -1.4481554   1.5931858  -1.8278202  -1.5751153  -0.01928964]
[-0.01287514 -1.3721259   1.5746301  -1.9434882  -1.5792477  -0.00877141]
[-3.9338181e-03 -1.2910470e+00  1.5476949e+00 -2.0557394e+00
 -1.5844587e+00  5.9030089e-04]
[ 0.00454562 -1.2037833   1.510621   -2.164815   -1.5906992   0.00883939]
[ 0.0116776  -1.1122956   1.46019    -2.2635028  -1.5973823   0.01507278]
[ 0.01833413 -1.0156705   1.3981818  -2.3564265  -1.6047064   0.02008578]
[ 0.0236902  -0.9147392   1.3196965  -2.436416   -1.6120366   0.02313502]
[ 0.02818281 -0.80857635  1.2255981  -2.5067809  -1.6194508   0.02460419]
[ 0.02726448 -0.795765    1.2131997  -2.513029   -1.6193823   0.02360758]
[ 0.02036208 -0.77704686  1.1937478  -2.5191767  -1.6160221   0.01755973]
[ 0.01172219 -0.75521994  1.1715642  -2.5279825  -1.6117204   0.01005257]
[ 1.7376429e-03 -7.

In [ ]:
        
robot.execute_arm_speed(init_joint, speed=1e-1)
robot.move_xc(np.array([0,0,0,0]), movetime=3)        

motor_array = np.load("planned_traj/scraps/motor_array.npy")[:,0,:] * 1.5
qs_array    = np.load("planned_traj/scraps/qs_array.npy") + np.array([0,0,0,-4*DEG90,0, DEG90])
# motor_array = np.array([[1000,0,0,0]]*30)
qs_array.shape, motor_array.shape

move_arm_soro(robot, qs_array, motor_array, time_array=np.ones(len(qs_array))*2)


In [41]:
# import numpy as np

motor_controls = np.load("planned_traj/scraps/motor_array.npy")[:,0,:] * 1.5
motor_controls.shape
robot.multiple_move_xc(motor_controls, movetimes=np.ones(len(motor_controls))*0.5 )

/tmp/ipykernel_119085/917136251.py:135: RuntimeWarning: divide by zero encountered in double_scalars
  unit_sleep = movetime/len(pos)


In [15]:
import numpy as np

qs_array = np.load("planned_traj/scraps/qs_array.npy")
qs_array[0] - init_joint


# init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90, 0]
# robot.execute_arm(init_joint, movetime = 5)

# for qs in qs_array:
#     robot.execute_arm(qs + np.array([0,0,0,-4*DEG90,0,-DEG90]), movetime = 10)

array([-3.78221267e-01, -4.00708568e-01,  2.98003205e-01,  6.38293777e+00,
        5.67389440e-04, -1.95196208e+00])

# Calibrate initpos
반드시 Cap을 벗기고 진행할것

In [ ]:
# Upright_joint = np.array([0, -90+45, 90-45, -90, 90, 90])/90*DEG90
# robot.execute_arm(Upright_joint)

In [ ]:
# robot.init_xc(initpos=np.array([None]))
# initpos = robot.xc.get_currpos()

In [ ]:
# relaxed_pos = initpos

In [ ]:
# initpos = np.array([2247, -546,  344, 2629, 1000, 2833, 1801, 4351, 4761, 2223, 1299,
#        1990,  2605, -703, 2454, 1732])
# # initpos[-4:] = relaxed_pos[-4:]
# robot.init_xc(initpos=initpos)

In [ ]:
# relaxed_pos

In [ ]:
# # 1번은 플러스, 2번 마이너스, 3번 마이너스, 4번 플러스
# pos = initpos + np.array([0,0,0,0, 0,0,0,0, 0,0,0,0, 100,200,200,200]) * np.array([1,-1,-1,1]*4)
# # robot.move_xc(pos)

# pos = pos + PROPORTION_MATRIX @ np.array([0,0,0,0, 0,0,0,0, 0,0,0,0, 600,600,600,600]) * np.array([1,-1,-1,1]*4)

# robot.move_xc(pos)

# robot.xc.get_currpos()

In [ ]:
# initpos = np.array([2247, -546,  344, 2629, 1000, 2833, 1801, 4351, 4761, 2223, 1299,
#        1990, 4110,  970, 3106, 3197])
# robot.init_xc(initpos)
# robot.xc.get_currpos()

# pos = initpos + np.array([300]*16) * np.array([1,-1,-1,1]*4)
# robot.move_xc(pos)
# time.sleep(2)

# pos = initpos + PROPORTION_MATRIX @ np.array([1000,0,0,0, 0,0,1000,0, 0,1000,0,0, 0,0,0,1000]) * np.array([1,-1,-1,1]*4)
# robot.move_xc(pos)
# time.sleep(2)


# pos = initpos + PROPORTION_MATRIX @ np.array([0,1000,0,0, 0,0,0,1000, 1000,0,0,0, 0,0,1000,0]) * np.array([1,-1,-1,1]*4)
# robot.move_xc(pos)
# time.sleep(2)

# robot.move_xc(initpos)

# Twist Open

In [7]:
robot.init_xc(np.array([None]))
initpos = robot.xc.get_currpos()
pos = initpos + np.array([1,1,1,1, 1,1,1,1, 1,1,1,1, 1,1,1,1])*1000 * np.array([1,-1,-1,1]*4)
robot.move_xc(pos)

initpos = np.array([2247, -546,  344, 2629, 1000, 2833, 1801, 4351, 4761, 2223, 1299,
       1990,  2705, -803, 2354, 1832])
robot.init_xc(initpos)

robot.xc.get_currpos()

[SOFTROBOT] CONNECT
 SUCCEEDED TO OPEN THE PORT!
 SUCCEEDED TO CHANGE THE BAUDRATE!
COMMUNICATION SUCCESS
[SOFTROBOT] DETECT ID
[SOFTROBOT] [16]MOTORS DETECTED 
[SOFTROBOT] ID_LIST:[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]]
TORQUE ON
No initpos Found
INITIALIZE POSITION
[1872  150  818 2040  574 3438 2144 3701 4350 3154 2029 1159 1867   54
 2864  908]
[SOFTROBOT] CONNECT
[PortHandler setup] The port number 0 has same device name... reinitialize port number 0!!
 SUCCEEDED TO OPEN THE PORT!
 SUCCEEDED TO CHANGE THE BAUDRATE!
COMMUNICATION SUCCESS
[SOFTROBOT] DETECT ID
[SOFTROBOT] [16]MOTORS DETECTED 
[SOFTROBOT] ID_LIST:[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]]
TORQUE ON
INITIALIZE POSITION
[2229 -515  367 2596  982 2861 1820 4319 4740 2264 1334 1954 2667 -762
 2390 1784]


array([2239, -532,  357, 2612,  991, 2848, 1812, 4332, 4750, 2242, 1316,
       1973, 2687, -784, 2374, 1808])

# Close Up

In [15]:
Target_up_joint = [0.001,-DEG90,DEG90,-DEG90,-DEG90,DEG90]

Target_down_joint = np.array([0.02,-86.79,106.58,-109.79, -90.04, 90.11])/90*DEG90

Target_down_joint2 = np.array([0.02,-87.48,104.85,-107.46, -90.03, 90.09])/90*DEG90
Target_down_joint = (Target_down_joint2+Target_down_joint)/2
assert (Target_up_joint * np.array([1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint * np.array([1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint2 * np.array([1,-1,1,-1,-1,1])>0).all()

robot.execute_arm(Target_up_joint)
robot.pmove_xc(np.array([0,0,0,0]*4))



Waiting for server...
Connected to server
Finish plan


In [16]:
coeff = 1.4
rollout_actuation = np.load("actuations/Custom_grasp.npy") * coeff

# Ungrasp
act = rollout_actuation[0].tolist()
robot.pmove_xc(np.array(act*4)/coeff)

In [17]:
# Ungrasp
act = rollout_actuation[0].tolist()
robot.pmove_xc(np.array(act*4)/coeff)

robot.execute_arm(Target_down_joint)

# 접근 동작
act = rollout_actuation[1].tolist()
robot.pmove_xc(np.array(act*4))

# 잡는 동작
act = rollout_actuation[2].tolist()
robot.pmove_xc(np.array(act*4))

# 절반 돌리고
act = rollout_actuation[3].tolist()
robot.pmove_xc(np.array(act*4))

# 나머지 돌리고
act = rollout_actuation[4].tolist()
robot.pmove_xc(np.array(act*4))

# 들고
act = rollout_actuation[5].tolist()
robot.pmove_xc(np.array(act*4))

# Ungrasp
act = rollout_actuation[0].tolist()
robot.pmove_xc(np.array(act*4)/coeff)

# 접근 동작
act = rollout_actuation[1].tolist()
robot.pmove_xc(np.array(act*4))

# 잡는 동작
act = rollout_actuation[2].tolist()
robot.pmove_xc(np.array(act*4))

# 절반 돌리고
act = rollout_actuation[3].tolist()
robot.pmove_xc(np.array(act*4))

# 나머지 돌리고
act = rollout_actuation[4].tolist()
robot.pmove_xc(np.array(act*4))

# 들고
act = rollout_actuation[5].tolist()
robot.pmove_xc(np.array(act*4))

# Ungrasp
act = rollout_actuation[0].tolist()
robot.pmove_xc(np.array(act*4)/coeff)

# 접근 동작
act = rollout_actuation[1].tolist()
robot.pmove_xc(np.array(act*4))

# 잡는 동작
act = rollout_actuation[2].tolist()
robot.pmove_xc(np.array(act*4))

# 절반 돌리고
act = rollout_actuation[3].tolist()
robot.pmove_xc(np.array(act*4))

# 나머지 돌리고
act = rollout_actuation[4].tolist()
robot.pmove_xc(np.array(act*4))



Waiting for server...
Connected to server
Finish plan


In [18]:
robot.execute_arm(Target_up_joint)
time.sleep(4)
robot.execute_arm(Target_down_joint2)


Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan


In [19]:
### Capping ###

# 놓고
act = rollout_actuation[3].tolist()
robot.pmove_xc(np.array(act*4))
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([0,2000,2000,0]*4))

# 잡는 동작
robot.pmove_xc(np.array([0,1800,200,800]*4))

# 절반 돌리고
robot.pmove_xc(np.array([500,500,200,1200]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([700,1000,200,2000]*4))

# 들고
robot.pmove_xc(np.array([3000,1000,0,2000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))


# 접근 동작
robot.pmove_xc(np.array([0,2000,2000,0]*4))

# 잡는 동작
robot.pmove_xc(np.array([0,1800,200,800]*4))

# 절반 돌리고
robot.pmove_xc(np.array([500,500,200,1200]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([700,1000,200,2000]*4))

# 들고
robot.pmove_xc(np.array([3000,1000,0,2000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))


# 접근 동작
robot.pmove_xc(np.array([0,2000,2000,0]*4))

# 잡는 동작
robot.pmove_xc(np.array([0,1800,200,800]*4))

# 절반 돌리고
robot.pmove_xc(np.array([500,500,200,1200]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([700,1000,200,2000]*4))

# 들고
robot.pmove_xc(np.array([3000,1000,0,2000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([0,2000,2000,0]*4))

# 잡는 동작
robot.pmove_xc(np.array([0,1800,200,800]*4))

# 절반 돌리고
robot.pmove_xc(np.array([500,500,200,1200]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([700,1000,200,2000]*4))

# 들고
robot.pmove_xc(np.array([3000,1000,0,2000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))


In [20]:
robot.execute_arm(Target_up_joint)

Waiting for server...
Connected to server
Finish plan


In [114]:
# # Ungrasp
# act = rollout_actuation[0].tolist()
# robot.pmove_xc(np.array(act*4)/coeff)

# # 접근 동작
# act = rollout_actuation[1].tolist()
# robot.pmove_xc(np.array(act*4))

# # 잡는 동작
# act = rollout_actuation[2].tolist()
# robot.pmove_xc(np.array(act*4))

# # 절반 돌리고
# act = rollout_actuation[3].tolist()
# robot.pmove_xc(np.array(act*4))

# # 나머지 돌리고
# act = rollout_actuation[4].tolist()
# robot.pmove_xc(np.array(act*4))

In [91]:
robot.pmove_xc(np.array([1000,0,1000,0]*4))

In [87]:
# # 잡는 동작
# robot.pmove_xc(np.array([0,1800,200,800]*4))

# # 절반 돌리고
# robot.pmove_xc(np.array([800,800,500,1500]*4))

# # 나머지 돌리고
# robot.pmove_xc(np.array([500,800,0,1800]*4))

In [53]:
coeff = 0.7
rollout_actuation = np.load("actuations/Custom_grasp.npy") * coeff

cap_actuation = np.zeros_like(rollout_actuation)

cap_actuation[:,0] = rollout_actuation[:,2]
cap_actuation[:,1] = rollout_actuation[:,3]
cap_actuation[:,2] = rollout_actuation[:,0]
cap_actuation[:,3] = rollout_actuation[:,1]


In [57]:
# # robot.execute_arm(Target_down_joint)

# # Ungrasp
# act = cap_actuation[0].tolist()
# robot.pmove_xc(np.array(act*4)/coeff)

# # robot.execute_arm(Target_down2_joint)

# # 접근 동작
# act = cap_actuation[1].tolist()
# robot.pmove_xc(np.array(act*4))

# # 잡는 동작
# act = cap_actuation[2].tolist()
# robot.pmove_xc(np.array(act*4))

# # 절반 돌리고
# act = cap_actuation[3].tolist()
# robot.pmove_xc(np.array(act*4))

# # 나머지 돌리고
# act = cap_actuation[4].tolist()
# robot.pmove_xc(np.array(act*4))

# # 들고
# act = cap_actuation[5].tolist()
# robot.pmove_xc(np.array(act*4))

# # Ungrasp
# act = cap_actuation[0].tolist()
# robot.pmove_xc(np.array(act*4)/coeff)


KeyboardInterrupt: 

In [46]:
# 놓고
act = rollout_actuation[0].tolist()
robot.pmove_xc(np.array(act*4)/coeff)

In [47]:
coeff = 1.5
rollout_actuation = np.load("actuations/Custom_grasp.npy") * coeff

# 놓고
act = rollout_actuation[0].tolist()
robot.pmove_xc(np.array(act*4)/coeff)

# robot.execute_arm(Target_down_joint)

# 들고
act = rollout_actuation[4].tolist()
robot.pmove_xc(np.array(act*4))

# # 원래대로 오고
# act = rollout_actuation[4].tolist()
# robot.pmove_xc(np.array(act*4))

# # 절반 돌리고
# act = rollout_actuation[3].tolist()
# robot.pmove_xc(np.array(act*4))

# # 나머지 돌리고
# act = rollout_actuation[2].tolist()
# robot.pmove_xc(np.array(act*4))

# # 들고
# act = rollout_actuation[1].tolist()
# robot.pmove_xc(np.array(act*4))

# # 놓고
# act = rollout_actuation[0].tolist()
# robot.pmove_xc(np.array(act*4)/coeff)


In [44]:
rollout_actuation = np.load("actuations/Custom_grasp.npy")
rollout_actuation = np.flip(rollout_actuation,axis=0)
rollout_actuation

array([[ 153, 1101, 1186,    8],
       [   0, 1712,  445,  909],
       [  85, 1025,  495, 1208],
       [  45,  867,  191, 1631],
       [1302,   63, -146, 1425],
       [1139,  137, 1054,    0]])

In [50]:

act = rollout_actuation[5].tolist()
robot.pmove_xc(np.array(act*4))

In [19]:
# robot.pmove_xc(np.array([0,0,0,0]*4))

# Jar

In [94]:
robot.pmove_xc(np.array([0,0,0,0]*4))

In [95]:
# robot.execute_arm(Place_up_joint)
# init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
# robot.execute_arm(init_joint)

In [114]:
Custom_grasp = np.load("actuations/Custom_grasp.npy")
Custom_grasp[0] = Custom_grasp[0]* 1.2
Custom_grasp[1:] = Custom_grasp[1:]* 1.6


Custom_grasp = Custom_grasp.tolist()
Custom_grasp

[[1366, 164, 1264, 0],
 [2083, 100, -233, 2280],
 [72, 1387, 305, 2609],
 [136, 1640, 792, 1932],
 [0, 2739, 712, 1454],
 [244, 1761, 1897, 12]]

In [115]:
robot.pmove_xc(np.tile(Custom_grasp[0],4))

In [113]:
for actuation in Custom_grasp:
    time.sleep(1)
    robot.pmove_xc(np.array(np.tile(actuation,4)))

for actuation in Custom_grasp:
    time.sleep(1)
    if actuation == Custom_grasp[-1]: break
    robot.pmove_xc(np.array(np.tile(actuation,4)))
    

In [63]:
actuation == Custom_grasp[-1]

True

In [48]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
Target_up_joint = np.array([-51.04,-79.42,73.24,-83.68,-89.61,81.53])/90*DEG90
Target_down_joint = np.array([-50.75,-75,100.9,-115.94,-89.4,82.05])/90*DEG90
Place_up_joint = np.array([-38.01,-54.74,37.94,-73.06,-89.60,94.45])/90*DEG90
Place_down_joint = np.array([-38.07,-49.32,77.91,-118.45,-89.65,94.72])/90*DEG90

assert (Target_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()


# robot.execute_arm(init_joint)

# robot.execute_arm(Target_up_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))

# robot.execute_arm(Target_down_joint)

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

# 들고
robot.pmove_xc(np.array([0,2000,3000,1000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# # 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

# robot.execute_arm(Target_up_joint)
# robot.execute_arm(Place_up_joint)
# robot.execute_arm(Place_down_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))
# robot.execute_arm(Place_up_joint)
# robot.execute_arm(init_joint)

# Tumbler

In [43]:
robot.pmove_xc(np.array([0,0,0,0]*4))

In [44]:
# robot.execute_arm(Place_up_joint)
# init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
# robot.execute_arm(init_joint)

In [45]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
Target_up_joint = np.array([-34.38,-90.85,93.59,-92.60,-89.61,98.27])/90*DEG90
Target_down_joint = np.array([-34.42,-87.67,107.73,-109.93,-89.65,98.36])/90*DEG90
Place_up_joint = np.array([-25.41,-66.39,63.93,-87.40,-89.59,107.17])/90*DEG90
Place_down_joint = np.array([-25.45,-56.45,89.79,-123.20,-89.68,107.37])/90*DEG90

assert (Target_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()


robot.execute_arm(init_joint)

robot.execute_arm(Target_up_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))

robot.execute_arm(Target_down_joint)

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

# 들고
robot.pmove_xc(np.array([0,2000,3000,1000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# # 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

robot.execute_arm(Target_up_joint)
robot.execute_arm(Place_up_joint)
robot.execute_arm(Place_down_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))
robot.execute_arm(Place_up_joint)
robot.execute_arm(init_joint)


Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan


# Tumbler2

In [46]:
robot.pmove_xc(np.array([0,0,0,0]*4))

In [47]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
Target_up_joint = np.array([-11.76,-94.52,93.83,-89.17,-89.60,120.91])/90*DEG90
Target_down_joint = np.array([-11.76,-91.46,111.03,-109.48,-89.64,121.01])/90*DEG90
Place_up_joint = np.array([-8.21,-66.55,61.03,-84.34,-89.52,124.35])/90*DEG90
Place_down_joint = np.array([-8.26,-57.02,90.09,-122.93,-89.67,124.56])/90*DEG90

assert (Target_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()


robot.execute_arm(init_joint)

robot.execute_arm(Target_up_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))

robot.execute_arm(Target_down_joint)

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

# 들고
robot.pmove_xc(np.array([0,2000,3000,1000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

# 들고
robot.pmove_xc(np.array([0,2000,3000,1000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))


robot.execute_arm(Target_up_joint)
robot.execute_arm(Place_up_joint)
robot.execute_arm(Place_down_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))
robot.execute_arm(Place_up_joint)
robot.execute_arm(init_joint)


Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan


## Gat

In [48]:
robot.pmove_xc(np.array([0,0,0,0]*4))


In [49]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
Target_up_joint   = np.array([16.84,-83.79,82.93,-89.01,-89.59,149.49])/90*DEG90
Target_down_joint = np.array([16.81,-82.13,96.38,-104.12,-89.61,149.57])/90*DEG90
Place_up_joint    = np.array([12.25,-55.41,42.79,-77.25,-89.58,144.78])/90*DEG90
Place_down_joint  = np.array([12.20,-48.98,77.21,-118.10,-89.63,145.01])/90*DEG90

assert (Target_up_joint * np.array([1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint * np.array([1,-1,1,-1,-1,1])>0).all()
assert (Place_up_joint * np.array([1,-1,1,-1,-1,1])>0).all()
assert (Place_down_joint * np.array([1,-1,1,-1,-1,1])>0).all()


robot.execute_arm(init_joint)

robot.execute_arm(Target_up_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))

robot.execute_arm(Target_down_joint)


# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1500,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,1200,1600]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))


robot.execute_arm(Target_up_joint)
robot.execute_arm(Place_up_joint)
robot.execute_arm(Place_down_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))
robot.execute_arm(Place_up_joint)
robot.execute_arm(init_joint)


Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan


In [50]:
robot.pmove_xc(np.array([0,0,0,0]*4))

# MAYO

In [17]:
pos = initpos+ PROPORTION_MATRIX @ np.array([1,0,1,0, 0,0,0,0, 0,0,0,0, 1,0,1,0]) * 4000 * np.array([1,-1,-1,1]*4)
pos = pos    + PROPORTION_MATRIX @ np.array([0,0,0,0, 0.3,0,1,0, 1,0,0,0.3, 0,0,0,0]) * 4000 * np.array([1,-1,-1,1]*4)

# pos = pos + np.array([0,0,0,0, 0,0,0,0, 0,0,0,400, 0,0,0,0]) * np.array([1,-1,-1,1]*4)
snapbot.set_goalposcluster(pos,1)

In [18]:
pos = initpos+ PROPORTION_MATRIX @ np.array([1,0,1,0, 0,0,0,0, 0,0,0,0, 1,0,1,0]) * 4000 * np.array([1,-1,-1,1]*4)
pos = pos    + PROPORTION_MATRIX @ np.array([0,0,0,0, 0.3,0,1,0, 1,0,0,0.3, 0,0,0,0]) * 4000 * np.array([1,-1,-1,1]*4)
pos = pos    + PROPORTION_MATRIX @ np.array([0,0,0,0, 0,0,1,0, 1,0,0,0, 0,0,0,0]) * 1000 * np.array([1,-1,-1,1]*4)

# pos = pos + np.array([0,0,0,0, 0,0,0,0, 0,0,0,400, 0,0,0,0]) * np.array([1,-1,-1,1]*4)
snapbot.set_goalposcluster(pos,1)